In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; float:center}</style>")

In [2]:
from __future__ import division, print_function

# from sys import argv
# import os
# import glob
# import math
import re
import pandas as pd
# import numpy as np
# import scipy as sp
# import scipy.stats as sps
# import matplotlib.pyplot as plt
# import seaborn as sns
# import simplejson as jsn
import requests 
from time import sleep
import json
import re
from urllib import urlretrieve, urlopen 
from bs4 import BeautifulSoup
import datetime
import time

# %matplotlib inline

# sns.set_context('notebook')
# sns.set_style('ticks')

In [3]:
def OpenPage(url):
    try: 
        html = urlopen(url)
    except HTTPError as e: 
        return None

    try:
        pageData = BeautifulSoup(html, 'lxml') #,"html.parser")
    except AttributeError as e: 
        return None
    return pageData


def TryFind(search):
    try: 
        result = search
    except (AttributeError, TypeError) as (e,te): 
        return None
    return result


def GetText(search):
    if search == None:
        return 'none'
    else: 
        try: 
            text = search.get_text().strip()
        except AttributeError as e: 
            return None
        return text
    
    
def GetLink(search):
    if search == None:
        return 'none'
    else:
        try: 
            link = search.attrs['href']
        except AttributeError as e:
            return None
        return link


def ScrapeSitemap(url):
    '''returns urls under <loc> tag in sitemap, as text string'''
    sitemap = OpenPage(url)
    taglist = sitemap.findAll('loc')
    urllist = [elem.text for elem in taglist]
    return urllist


def URLFilter(urllist, expression):
    '''filters url list via regex to eliminate non-recipe pages (category aggregations, misc posts, etc)'''
    matches = []
    for url in urllist: 
        match = re.match(expression, url)
        if match: 
            matches += [match.group(0)]
    return matches

In [5]:
def CommentData_SK(comment, ID, children, child_id):
    '''get the data from each comment - '''
    comment_info = {}

    name = TryFind(comment.find('b',{'class':'fn author-name'}))
    if TryFind(name.find('a')) == None:
        comment_info['username'] = GetText(TryFind(comment.find('b',{'class':'fn author-name'})))
        comment_info['usersite'] = 'none'

    else: 
        comment_info['username'] = TryFind(comment.find('a')).contents[0]
        comment_info['usersite'] = TryFind(comment.find('a')).attrs['href']

        
    comment_info['usercomment'] = GetText(TryFind(comment.find('div',{'class':'comment-content'}).find('p')))
    comment_info['children'] = children
    comment_info['child_id'] = child_id
    comment_info['commentID'] = ID

    return comment_info


def GetComments_SK(soup):
    '''get comments and their place in the hierarchy'''

    comments_data = {}

    allcomments = TryFind(soup.find('ol',{'class':'comment-list'}))
    number_of_comments = len(allcomments.findAll('article',{'class':'comment-body'}))

    comment = allcomments.find('article',{'class':'comment-body'})

    for comment_ID in range(0, number_of_comments):

        nextComment = comment.findNext('article',{'class':'comment-body'})

#         # look for parents
#         if comment_ID!= 1: 
#             if comment.parent.parent['class'][0] == 'children':
#                 comment_has_parent, parent_id = 'yes', 0
#         else: comment_has_parent, parent_id = 'no', 0


        # look for children
        if comment_ID < number_of_comments-1:
            if nextComment.parent.parent['class'][0] == 'children':
                comment_has_children, child_id = 'yes', comment_ID+1
            else: comment_has_children, child_id = 'no', 0 

        comments_data[comment_ID] = CommentData_SK(comment, comment_ID, comment_has_children, child_id)

        comment = comment.findNext('article')

    return comments_data, number_of_comments

In [4]:
url = 'https://smittenkitchen.com'
sk_sitemap = 'https://smittenkitchen.com/sitemap.xml'

# for SK, recipes are listed in <url>/yr/mo/<name> format, other aggregations/lists do not have this same formatting 
sk_regex = 'https://smittenkitchen.com/[0-9]{3,}.*'


urls_sk = URLFilter(ScrapeSitemap(sk_sitemap), sk_regex)

In [7]:
url = urls_sk[0]
soup = OpenPage(url)

allcomments = TryFind(soup.find('ol',{'class':'comment-list'}))
number_of_comments = len(allcomments.findAll('article',{'class':'comment-body'}))

comment = allcomments.find('article',{'class':'comment-body'})


In [13]:
a = comment.find('footer',{'class':'comment-footer comment-metadata'}).find('time',{'class':'comment-time'})

In [20]:
time.

<time class="comment-time">\n<a href=" https://smittenkitchen.com/2010/04/classic-cobb-salad/#comment-707336">\n<span class="date">April 14, 2010</span>\n\t\t\t\t\t\tat\t\t\t\t\t\t<span class="time">10:14 pm</span>\n</a>\n</time>